In [ ]:
import sys
sys.path.append('../')

import numpy as np
import matplotlib.pyplot as plt
import imageio

import yaml
import imageruler
import mesher
import utils
import plot_utils
import skfmm

import matplotlib.colors as mcolors
from matplotlib.ticker import FormatStrFormatter

_Ext = utils.Extent
plt.rcParams.update(plot_utils.high_res_plot_settings)

In [ ]:
with open("./settings.yaml", "r") as file:
  config = yaml.safe_load(file)

mesh_cfg = config['STAMP_MESH']
bbox_cfg = config['STAMP_BBOX']

# Define Stamp Mesh

In [ ]:
stamp_bbox = mesher.BoundingBox(x=_Ext(bbox_cfg['x_min'], bbox_cfg['x_max']),
                                y=_Ext(bbox_cfg['y_min'], bbox_cfg['y_max'])
                                )
stamp_mesh = mesher.Mesher(nelx=mesh_cfg['nelx'],
                           nely=mesh_cfg['nely'],
                           bounding_box=stamp_bbox)

# Define the generators

In [ ]:
circ_gen = np.array(imageio.imread("../data/circle.png"))
rounged_sq = np.array(imageio.imread("../data/square.png"))
rounded_t_shape = np.array(imageio.imread("../data/T_shape.png"))
rounded_c_shape = np.array(imageio.imread("../data/C_shape.png"))
rounded_i_shape = np.array(imageio.imread("../data/I_shape.png"))
rounded_x_shape = np.array(imageio.imread("../data/X_shape.png"))
rounded_l_shape = np.array(imageio.imread("../data/L_shape.png"))
hexagon = np.array(imageio.imread("../data/hexagon.png"))
ellipse_1 = np.array(imageio.imread("../data/ellipse_1.png"))
ellipse_2 = np.array(imageio.imread("../data/ellipse_2.png"))
slot = np.array(imageio.imread("../data/slot.png"))
triangle = np.array(imageio.imread("../data/triangle.png"))
trapezium = np.array(imageio.imread("../data/trapezium.png"))
pentagon = np.array(imageio.imread("../data/pentagon.png"))
rhombus = np.array(imageio.imread("../data/rhombus.png"))


In [ ]:
shapes = []

shapes.append(circ_gen)
shapes.append(rounged_sq)
shapes.append(rounded_t_shape)
shapes.append(rounded_c_shape)
shapes.append(rounded_i_shape)
shapes.append(rounded_x_shape)
shapes.append(rounded_l_shape)
shapes.append(hexagon)
shapes.append(pentagon)
shapes.append(slot)
shapes.append(rhombus)
shapes.append(trapezium)
shapes.append(triangle)
shapes.append(ellipse_1)
shapes.append(ellipse_2)


# Convert Stamp images to SDF

In [ ]:
def calculate_shape_sdf(shape: np.ndarray, dx: float) -> np.ndarray:
  """
  Calculate the signed distance field (SDF) of a given shape.
  
  Args:
    shape: Array of the shapes with density values of shape (mesh.nelx, mesh.nely, 4). 4 represents the number of picture channels.
    dx: Grid spacing.

  Returns: 
    A float array indicating the SDF of the shape.
  """
  im = np.int64(np.any(shape[:, :, :3], axis=2))
  shape_density = np.where(im, 0, -1) + 0.5
  return skfmm.distance(-shape_density, dx=dx)

In [ ]:
def plot_sdf(sdf: np.ndarray, solid_mfs: float, void_mfs: float) -> None:
  """
  Plot the signed distance field (SDF) with color bar and annotations.

  Args:
    sdf: Array of the SDF of shape (mesh.nelx, mesh.nely) to plot.
    solid_mfs: Float indicating solid minimum feature size.
    void_mfs: Float indicating void minimum feature size.
  """
  min_sdf = sdf.min()
  max_sdf = sdf.max()
  norm = mcolors.TwoSlopeNorm(vmin=min_sdf, vcenter=0, vmax=max_sdf)
  
  plt.figure()
  img = plt.imshow(sdf, cmap='coolwarm', origin='lower', norm=norm)
  cbar = plt.colorbar(img)
  cbar.set_ticks(np.linspace(min_sdf, max_sdf, num=5))
  cbar.ax.tick_params(labelsize=16)
  cbar.ax.yaxis.set_major_formatter(FormatStrFormatter('%d'))
  plt.axis('off')
  plt.title(f'solid_mfs: {solid_mfs:.2f}, void_mfs: {void_mfs:.2f}')
  plt.show()

In [ ]:
def process_shape_images(shapes: list, stamp_mesh: mesher.Mesher, stamp_bbox: mesher.BoundingBox) -> np.ndarray:
  """
  Process a list of shapes to compute their signed distance fields (SDF) and plot them.

  Args:
    shapes: List of arrays representing shapes.
    stamp_mesh: A dataclass object containing stamp mesh information.
    stamp_bbox: A dataclass object containing bounding box dimensions.

  Returns: 
    An array of SDFs for each shape.
  """
  train_data = np.zeros((len(shapes), stamp_mesh.nelx, stamp_mesh.nely, 1))

  for ctr, shape in enumerate(shapes):
    dx = stamp_bbox.lx / shape.shape[0]
    
    sdf = calculate_shape_sdf(shape, dx)
    train_data[ctr, :, :, 0] = sdf
    
    solid_mfs_px, void_mfs_px = imageruler.minimum_length_scale(sdf > 0)
    solid_mfs = solid_mfs_px * stamp_mesh.elem_size[0]
    void_mfs = void_mfs_px * stamp_mesh.elem_size[0]

    plot_sdf(sdf, solid_mfs, void_mfs)
  
  return train_data

In [ ]:
train_data = process_shape_images(shapes, stamp_mesh, stamp_bbox)

# Save Stamp SDF

In [ ]:
np.save('../data/train_sdf_images.npy', train_data)